In [ ]:
# @title Conectare la drive
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# @title Imports

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import tensorflow.keras.backend as K
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

# Dataset

In [ ]:
# @title Setari cale

train_img_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/img/cls"
train_bin_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/bin_mask/cls"
train_mult_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/mult_mask/cls"

validation_img_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/validation/img/cls"
validation_bin_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/validation/bin_mask/cls"
validation_mult_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/validation/mult_mask/cls"


In [ ]:
# @title Incarcam path-urile

def load_paths(imd_dir, mask_dir):
  image_paths = sorted([os.path.join(imd_dir, file_name) for file_name in os.listdir(imd_dir) if file_name.endswith(".tif")])
  mask_paths = sorted([os.path.join(mask_dir, file_name) for file_name in os.listdir(mask_dir) if file_name.endswith(".tif")])
  return image_paths, mask_paths

train_img_paths, train_mask_paths = load_paths(train_img_path, train_mult_mask_path)

validation_img_paths, validation_mask_paths = load_paths(validation_img_path, validation_mult_mask_path)

In [ ]:
# @title Augumentare data

def augment_data(image, mask):
  # Random flip orizontal
  if tf.random.uniform(()) > 0.5:
    image = tf.image.flip_left_right(image)
    mask = tf.image.flip_left_right(mask)

  # Random flip vertical
  if tf.random.uniform(()) > 0.5:
    image = tf.image.flip_up_down(image)
    mask = tf.image.flip_up_down(mask)

  # Random rotire
  angle = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
  image = tf.image.rot90(image, angle)
  mask = tf.image.rot90(mask, angle)

  # Random zoom
  if tf.random.uniform(()) > 0.5:
    scales = list(np.arange(0.8, 1.0, 0.01))
    scale = np.random.choice(scales)
    new_height = tf.cast(1040 * scale, tf.int32)
    new_width = tf.cast(1392 * scale, tf.int32)

    image = tf.image.resize(image, (new_height, new_width))
    mask = tf.image.resize(mask, (new_height, new_width), method='nearest')

    image = tf.image.resize_with_crop_or_pad(image, 1040, 1392)
    mask = tf.image.resize_with_crop_or_pad(mask, 1040, 1392)

  return image, mask

In [ ]:
# @title Preprocesare imagine + masca

def load_tif_with_pil(path):
  img = Image.open(path.numpy().decode('utf-8'))
  img = img.convert('L') # Convertim la grayscale
  img = np.array(img, dtype=np.float32)
  return img

def preprocess_bin(image_path, mask_path):
  # Citim imaginea
  image = tf.py_function(func=load_tif_with_pil, inp=[image_path], Tout=tf.float32)
  image = tf.expand_dims(image, axis=-1)
  image.set_shape([1040, 1392, 1])

  # Citim masca
  mask = tf.py_function(func=load_tif_with_pil, inp=[image_path], Tout=tf.float32)
  mask = tf.expand_dims(mask, axis=-1)
  mask.set_shape([1040, 1392, 1])

  # Normalizare si
  image = tf.cast(image, tf.float32) / 255.0
  mask = tf.cast(mask, tf.float32) / 255.0

  # image.set_shape([1040, 1392, 1])
  # mask.set_shape([1040, 1392, 1])

  return image, mask

def preprocess_mult(image_path, mask_path):
  # Citim imaginea
  image = tf.py_function(func=load_tif_with_pil, inp=[image_path], Tout=tf.float32)
  image = tf.expand_dims(image, axis=-1)
  image.set_shape([1040, 1392, 1])

  # Citim masca
  mask = tf.py_function(func=load_tif_with_pil, inp=[mask_path], Tout=tf.float32)
  mask = tf.expand_dims(mask, axis=-1)
  mask.set_shape([1040, 1392, 1])

  # Normalizare
  image = tf.cast(image, tf.float32) / 255.0
  mask = tf.cast(mask, tf.float32)

  return image, mask

In [ ]:
# @title Creare dataset-ului

def create_dataset(image_paths, mask_paths, batch_size=2, shuffle=True, augment=False):
  dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
  dataset = dataset.map(preprocess_mult, num_parallel_calls=tf.data.AUTOTUNE)

  if augment:
    dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=100)

  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

train_dataset = create_dataset(train_img_paths, train_mask_paths, augment=True)
validation_dataset = create_dataset(validation_img_paths, validation_mask_paths)

In [ ]:
# @title Verificare shape imagini + masca

for images, masks in validation_dataset.take(1):
  print("Shape imagini batch:", images.shape)
  print("Shape măști batch:", masks.shape)


Shape imagini batch: (2, 1040, 1392, 1)
Shape măști batch: (2, 1040, 1392, 1)


# Model CNN Simplu Binar

In [ ]:
# @title Model CNN simplu

from tensorflow.keras import layers, models

def build_simple_cnn(input_shape=(1040, 1392, 1)):
  input = layers.Input(shape=input_shape)

  # Encoder
  x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input)
  x = layers.MaxPooling2D((2, 2), padding='same')(x)

  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = layers.MaxPooling2D((2, 2), padding='same')(x)

  # Decoder
  x = layers.Conv2DTranspose(32, (2, 2), strides=2, activation='relu', padding='same')(x)
  x = layers.Conv2DTranspose(16, (2, 2), strides=2, activation='relu', padding='same')(x)

  # Strat de iesire
  outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

  model = models.Model(inputs=input, outputs=outputs)
  return model

In [ ]:
# @title Metrici custom

import tensorflow.keras.backend as K

# Intersection over Union (IoU)
def iou(y_true, y_pred):
  y_pred = K.cast(y_pred > 0.5, 'float32')
  intersection = K.sum(y_true * y_pred)
  union = K.sum(y_true) + K.sum(y_pred) - intersection
  return intersection / (union + K.epsilon())

def dice(y_true, y_pred):
  y_pred = K.cast(y_pred > 0.5, 'float32')
  intersection = K.sum(y_true * y_pred)
  dice = (2. * intersection) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())
  return dice

In [ ]:
# @title Construire si compilare a modelului

simple_CNN = build_simple_cnn()

simple_CNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou, dice])

simple_CNN.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1040, 1392, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 1040, 1392, 16) │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 520, 696, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 520, 696, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 260, 348, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 520, 696, 32)   │         4,128 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 1040, 1392, 16) │         2,064 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1040, 1392, 1)  │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,009 (43.00 KB)

 Trainable params: 11,009 (43.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# @title Antranare model

train = simple_CNN.fit(train_dataset, validation_data=validation_dataset, epochs=5)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipyk

In [ ]:
for images, masks in validation_dataset.take(1):
  print("Pixelii unici în ground truth:", tf.unique(tf.reshape(masks, [-1]))[0].numpy())

Pixelii unici în ground truth: [0. 1. 4. 5. 2.]


In [ ]:
from PIL import Image
import numpy as np

# Exemplu: deschidem un mask .tif brut
mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/train/bin_mask/cls/0001.tif"  # înlocuiește cu un fișier real
mask_image = Image.open(mask_path)
mask_array = np.array(mask_image)

print("Valori unice în mască:", np.unique(mask_array))


Valori unice în mască: [  0 255]


In [ ]:
# import matplotlib.pyplot as plt

# for images, masks in validation_dataset.take(1):
#     preds = simple_CNN.predict(images)
#     preds = (preds > 0.5).astype(np.float32)  # threshold la 0.5

#     plt.figure(figsize=(6, 20))
#     for i in range(2):  # 2 imagini
#         plt.subplot(3, 2, 1+i)
#         plt.imshow(tf.squeeze(images[i]), cmap='gray')
#         plt.title('Imagine originală')

#         plt.subplot(3, 2, 3+i)
#         plt.imshow(tf.squeeze(masks[i]), cmap='gray')
#         plt.title('Ground truth (etichete)')

#         plt.subplot(3, 2, 5+i)
#         plt.imshow(tf.squeeze(preds[i]), cmap='gray')
#         plt.title('Predicție model')
#     plt.show()

simple_CNN.evaluate(validation_dataset)

50/50 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.7377 - dice: 0.0812 - iou: 0.0436 - loss: 0.6916


[0.6916448473930359,
 0.7391826510429382,
 0.04576931148767471,
 0.08556623756885529]

# Model CNN Simplu Multi-Clasa

In [ ]:
# @title Model Multi-Clasa

from tensorflow.keras import layers, models

def build_simple_cnn_multiclass(input_shape=(1040, 1392, 1), num_classes=8):
  input = layers.Input(shape=input_shape)

  # Encoder
  x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input)
  x = layers.MaxPooling2D((2, 2))(x)

  x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = layers.MaxPooling2D((2, 2))(x)

  # Decoder
  x = layers.Conv2DTranspose(32, (2, 2), strides=2, activation='relu', padding='same')(x)
  x = layers.Conv2DTranspose(16, (2, 2), strides=2, activation='relu', padding='same')(x)

  # Output
  outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(x)

  model = models.Model(inputs=input, outputs=outputs)
  return model

In [ ]:
# @title Metrici personalizati

import tensorflow.keras.backend as K

def iou_multiclass(y_true, y_pred):
  # Convertim predictia softmax in argmax
  y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

  y_true = tf.squeeze(y_true, axis=-1) # Daca masca are inca un canal 1
  y_true = tf.cast(y_true, tf.int32)

  intersection = tf.reduce_sum(tf.cast(tf.equal(y_true, y_pred), tf.float32))
  union = tf.reduce_sum(tf.ones_like(y_true, dtype=tf.float32))

  return intersection / (union + K.epsilon())

def dice_multiclass(y_true, y_pred):
  # Convertim predictia softmax in argmax
  y_pred = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

  y_true = tf.squeeze(y_true, axis=-1) # Daca masca are inca un canal 1
  y_true = tf.cast(y_true, tf.int32)

  y_true_one_hot = tf.one_hot(y_true, depth=8)
  y_pred_one_hot = tf.one_hot(y_pred, depth=8)

  intersection = tf.reduce_sum(y_true_one_hot * y_pred_one_hot)
  denominator = tf.reduce_sum(y_true_one_hot) + tf.reduce_sum(y_pred_one_hot)

  return 2.0 * intersection / (denominator + K.epsilon())

In [ ]:
# @title Compilam modelul

multiclass_CNN = build_simple_cnn_multiclass()

multiclass_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["sparse_categorical_accuracy", iou_multiclass, dice_multiclass])

multiclass_CNN.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1040, 1392, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1040, 1392, 16) │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 520, 696, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 520, 696, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 260, 348, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 520, 696, 32)   │         4,128 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 1040, 1392, 16) │         2,064 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 1040, 1392, 8)  │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,128 (43.47 KB)

 Trainable params: 11,128 (43.47 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# @title Antrenare

train = multiclass_CNN.fit(train_dataset, validation_data=validation_dataset, epochs=5)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipyk

In [ ]:
# @title Testam modelul

test_img_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/test/img/cls"
test_mask_path = "/content/drive/MyDrive/Facultate/Licenta/ds1/test/mult_mask/cls"

test_imgs, test_masks = load_paths(test_img_path, test_mask_path)

test_dataset = create_dataset(test_imgs, test_masks, shuffle=False)

results = multiclass_CNN.evaluate(test_dataset)
print("Test loss:", results[0])
print("Test accuracy:", results[1])
print("Test IoU:", results[2])
print("Test Dice:", results[3])

50/50 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - dice_multiclass: 0.0366 - iou_multiclass: 0.0366 - loss: 2.0776 - sparse_categorical_accuracy: 0.0366
Test loss: 2.077559471130371
Test accuracy: 0.037519995123147964
Test IoU: 0.037519995123147964
Test Dice: 0.037519995123147964


# Vizualizare

In [ ]:
# @title Colorare masca

palette = np.array([
  [0, 0, 0],       # 0: Fundal - Negru
  [255, 0, 0],     # 1: Rod (bacterie) - Rosu
  [0, 255, 0],     # 2: RBC/WBC - Verde
  [0, 0, 255],     # 3: Yeast - Albastru
  [255, 255, 0],   # 4: Miscellaneous - Galben
  [255, 0, 255],   # 5: Single EPC - Magenta
  [0, 255, 255],   # 6: Small EPC sheet - Cyan
  [255, 255, 255]  # 7: Large EPC sheet - Alb
], dtype=np.uint8)

In [ ]:
# @ Vizualizare imagine + ground truth + predictie

import os

# 1. Creăm directorul pentru salvare dacă nu există
output_dir = "/content/outputs_test"
os.makedirs(output_dir, exist_ok=True)

# 2. Funcție color map (o refacem puțin robustă)
def apply_color_map(mask):
    mask = tf.squeeze(mask, axis=-1)
    mask = tf.cast(mask, tf.int32)
    mask = mask.numpy()
    color_mask = palette[mask]
    return color_mask

# 3. Parcurgem batch-uri și salvăm imaginile
for batch_idx, (images, masks) in enumerate(test_dataset.take(5)):  # luam 5 batch-uri
    preds = multiclass_CNN.predict(images)
    preds = tf.argmax(preds, axis=-1)
    preds = tf.expand_dims(preds, axis=-1)

    batch_size = images.shape[0]

    for i in range(batch_size):
        plt.figure(figsize=(24, 8))

        # Imagine originală
        plt.subplot(1, 3, 1)
        plt.imshow(tf.squeeze(images[i]), cmap='gray')
        plt.title('Imagine originală')
        plt.axis('off')

        # Ground Truth colorat
        plt.subplot(1, 3, 2)
        gt_color = apply_color_map(masks[i])
        plt.imshow(gt_color)
        plt.title('Ground Truth')
        plt.axis('off')

        # Predicție multiclass_CNN colorată
        plt.subplot(1, 3, 3)
        pred_color = apply_color_map(preds[i])
        plt.imshow(pred_color)
        plt.title('Predicție multiclass_CNN')
        plt.axis('off')

        plt.tight_layout()

        # Salvăm imaginea
        save_path = os.path.join(output_dir, f"batch{batch_idx}_img{i}.png")
        plt.savefig(save_path)
        plt.close()  # închidem figura ca să nu consume memorie

print(f"Salvat imaginile în {output_dir}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Salvat imaginile în /content/outputs_test 🎯


In [ ]:
for images, masks in train_dataset.take(1):
  print("Shape imagini:", images.shape)
  print("Shape măști:", masks.shape)
  print("Valori unice în măști:", tf.unique(tf.reshape(masks, [-1]))[0].numpy())

  print(multiclass_CNN.summary())
  print(multiclass_CNN.loss)
  print(multiclass_CNN.metrics_names)

  preds = multiclass_CNN.predict(images)
  preds = tf.argmax(preds, axis=-1)


Shape imagini: (2, 1040, 1392, 1)
Shape măști: (2, 1040, 1392, 1)
Valori unice în măști: [0. 2. 5.]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1040, 1392, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 1040, 1392, 16) │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 520, 696, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 520, 696, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 260, 348, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 520, 696, 32)   │         4,128 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 1040, 1392, 16) │         2,064 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1040, 1392, 8)  │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,386 (130.42 KB)

 Trainable params: 11,128 (43.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,258 (86.95 KB)

None
sparse_categorical_crossentropy
['loss', 'compile_metrics']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


#  Model U-Net Multi-Clasa

In [ ]:
# @title U-Net

def conv_block(inputs, num_filters):
  x = layers.Conv2D(num_filters, (3,3), padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4))(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv2D(num_filters, (3,3), padding="same", kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(1e-4))(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  return x

def build_unet_multiclass(input_shape=(256,256,1), num_classes=8):
  inputs_tensor = layers.Input(shape=input_shape)

  # ENcoder
  c1 = conv_block(inputs_tensor, 32)
  p1 = layers.MaxPooling2D((2,2))(c1)

  c2 = conv_block(p1, 64)
  p2 = layers.MaxPooling2D((2,2))(c2)

  c3 = conv_block(p2, 128)
  p3 = layers.MaxPooling2D((2,2))(c3)

  c4 = conv_block(p3, 256)
  p4 = layers.MaxPooling2D((2,2))(c4)

  # Bottleneck
  c5 = conv_block(p4, 512)

  # Decoder
  u6 = layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding="same")(c5)
  u6 = layers.concatenate([u6, c4])
  c6 = conv_block(u6, 256)

  u7 = layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c6)
  u7 = layers.concatenate([u7, c3])
  c7 = conv_block(u7, 256)

  u8 = layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c7)
  u8 = layers.concatenate([u8, c2])
  c8 = conv_block(u8, 128)

  u9 = layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding="same")(c8)
  u9 = layers.concatenate([u9, c1])
  c9 = conv_block(u9, 64)

  # Output
  outputs = layers.Conv2D(num_classes, (1,1), activation="softmax")(c9)

  model = models.Model(inputs=[inputs_tensor], outputs=[outputs])
  return model

In [ ]:
# @title Loss

def dice_loss_multiclass(y_true, y_pred, smooth=1e-6): # smooth=1e-6 pentru Dice Loss previne diviziunea la zero
  y_true = tf.squeeze(y_true, axis=-1) # Daca masca are inca un canal 1
  y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=8)
  y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7) # pentru stabilitate numerica intre cele doua valori

  intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2])
  union = tf.reduce_sum(y_true + y_pred, axis=[1,2])

  dice = (2.0 * intersection + smooth) / (union + smooth)
  dice_loss = 1 - dice

  return tf.reduce_mean(dice_loss)

def combined_loss(y_true, y_pred):
  scce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred) # Clasifica corect pixel cu pixel
  dice_loss = dice_loss_multiclass(y_true, y_pred) # Cat de bine se suprapun regiunile segmentate
  return scce + dice_loss

In [ ]:
# @title Creare Dataset special pentru U-Net
# Selectam bucati random 256x256 exact ca in research paper

def extract_random_patch(image, mask, patch_size=256):
  image_height = tf.shape(image)[0]
  image_width = tf.shape(image)[1]

  # Alegem random un punct de start (y, x)
  #  Eroare: max_y si max_x pot fi negative
  #  Solutie: Verificare daca dimensiunile sunt mai mici decat patch_size
  max_y = tf.maximum(0, image_height - patch_size)
  max_x = tf.maximum(0, image_width - patch_size)

  start_y = tf.random.uniform((), 0, max_y + 1, dtype=tf.int32)
  start_x = tf.random.uniform((), 0, max_x + 1, dtype=tf.int32)

  # Extragem patch-ul
  image_patch = tf.image.crop_to_bounding_box(image, start_y, start_x, patch_size, patch_size)
  mask_patch = tf.image.crop_to_bounding_box(mask, start_y, start_x, patch_size, patch_size)

  return image_patch, mask_patch

def create_patch_dataset(img_paths, mask_paths, batch_size=8, shuffle=True, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((img_paths, mask_paths))

    dataset = dataset.map(preprocess_mult, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=100)

    def extract_multiple_patches(img, mask):
      img_patches = []
      mask_patches = []

      for _ in range(3):  # extragem 3 patch-uri
        img_patch, mask_patch = extract_random_patch(img, mask)
        img_patches.append(img_patch)
        mask_patches.append(mask_patch)

      return tf.data.Dataset.from_tensor_slices((img_patches, mask_patches))

    dataset = dataset.flat_map(extract_multiple_patches)

    if augment:
        dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

train_dataset_unet = create_patch_dataset(train_img_paths, train_mask_paths, augment=True)
validation_dataset_unet = create_patch_dataset(validation_img_paths, validation_mask_paths)

print(train_dataset_unet.cardinality().numpy())
print(validation_dataset_unet.cardinality().numpy())

-2
-2


In [ ]:
unet_model = build_unet_multiclass()

unet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=combined_loss, metrics=["sparse_categorical_accuracy", iou_multiclass, dice_multiclass])

unet_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_57 (Conv2D)  │ (None, 256, 256,  │        320 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_57[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_54 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_58 (Conv2D)  │ (None, 256, 256,  │      9,248 │ re_lu_54[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        128 │ conv2d_58[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_55 (ReLU)     │ (None, 256, 256,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 128, 128,  │          0 │ re_lu_55[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_59 (Conv2D)  │ (None, 128, 128,  │     18,496 │ max_pooling2d_12… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_59[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_56 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 128, 128,  │     36,928 │ re_lu_56[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_60[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_57 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 64, 64,    │          0 │ re_lu_57[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_13… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_61[0][0] 

 Total params: 8,782,664 (33.50 MB)

 Trainable params: 8,775,880 (33.48 MB)

 Non-trainable params: 6,784 (26.50 KB)

In [ ]:
# @title Antrenare

train = unet_model.fit(
    train_dataset_unet,
    validation_data=validation_dataset_unet,
    epochs=50,
    steps_per_epoch=200,
    validation_steps=50,
    verbose=1
  )

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_204']
Received: inputs=Tensor(shape=(None, None, None, 1))
  warnings.warn(msg)


InvalidArgumentError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipyk